In [ ]:
import numpy as np
import sys
import os
import pickle as pkl

from astropy import units as u
from astropy.time import Time
from astropy.table import vstack

import starry

sys.path.append("../volcano")
from utils import get_body_ephemeris

np.random.seed(42)

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
%matplotlib inline
%run notebook_setup.py

In [ ]:
# Create a Starry map and place the two volcanos on the surface of Io
ydeg = 30
map = starry.Map(ydeg)

map.add_spot(amp=1e-01, sigma=0.02, lat=-12.0, lon=340)
map.add_spot(amp=1e-02, sigma=0.02, lat=20.5, lon=60.0)

fig, ax = plt.subplots(figsize=(8, 4))
map.show(ax=ax, projection="rect")

In [ ]:
# Pick some random observation times in a 1yr period
start = Time("2019-01-01", format="isot")
stop = Time("2020-01-01", format="isot")
start_times = np.sort(np.random.uniform(start.mjd, stop.mjd, 100))

eph_list_io = []
eph_list_jupiter = []

for i, t in enumerate(start_times):
    # Assume observations in a 2 day range
    delta_t = 1.0  # interval [days]
    npts = delta_t * 24 * 60 * 30  # 2 sec cadence
    times = Time(np.linspace(t, t + delta_t, int(npts)), format="mjd")

    # Compute ephemeris
    eph_io = get_body_ephemeris(
        times, body_id="501", step="1m", return_orientation=True
    )
    eph_jup = get_body_ephemeris(
        times, body_id="599", step="1m", return_orientation=False
    )

    eph_list_io.append(eph_io)
    eph_list_jupiter.append(eph_jup)

In [ ]:
# Save useful data
map_imgs = []  # map in pixel space for visualization later
coeff_list = []  # save SH coeff_list for each lightcurve
amps_list = []  # map amplitudes
A_list = []  # design matrix for each lihgtcurve
amp_spot1 = []  # spot 1 amplitude for each lightcurve
amp_spot2 = []
t_spot = []  # times at which the spot amplitudes were evaluated
lcs_true = []  # modeled flux
times_list = []  # observation times

# Periodicities of the spots
P1 = 23.0  # days
P2 = 67.0

# Iterate over lightcurves
for i in range(len(eph_list_io)):
    eph_io = eph_list_io[i]
    eph_jup = eph_list_jupiter[i]

    # Convert everything to units where the radius of Io = 1
    radius_jup = eph_jup["ang_width"] / eph_io["ang_width"]
    rel_ra = (eph_jup["RA"] - eph_io["RA"]).to(u.arcsec) / (
        0.5 * eph_io["ang_width"].to(u.arcsec)
    )
    rel_dec = (eph_jup["DEC"] - eph_io["DEC"]).to(u.arcsec) / (
        0.5 * eph_io["ang_width"].to(u.arcsec)
    )

    obl = np.array(eph_io["obl"])
    inc = np.array(eph_io["inc"])
    theta = np.array(eph_io["theta"])
    xo = np.array(-rel_ra)
    yo = np.array(rel_dec)
    zo = np.ones(len(yo))
    ro = np.array(radius_jup)

    # Get only the times when Jupiter is occulting Io in eclipse or Io
    # is in eclipse
    mask1 = np.any([eph_io["occ_umbra"], eph_io["ecl_tot"]], axis=0)
    mask2 = np.sqrt(xo ** 2 + yo ** 2) < (
        1 + 0.9 * ro
    )  # avoid times when Io is behind Jupiter

    mask = np.all([mask1, ~mask2], axis=0)

    eph_io = eph_io[mask]
    eph_jup = eph_jup[mask]

    # Initialize map
    map.reset()

    # Time dependent amplitudes of the spots
    t = np.mean(eph_io.time.mjd)

    a1 = 0.01 + 1e-02 * np.sin(t / P1)
    a2 = 0.01 + 1e-02 * np.cos(t / P2)

    map.add_spot(amp=a1, sigma=0.01, lat=-12.0, lon=340)
    map.add_spot(amp=a2, sigma=0.01, lat=20, lon=40.0)

    # Fix obliquity and inclination for given lc
    map.inc = np.mean(inc[mask])
    map.obl = np.mean(obl[mask])
    ro = np.mean(ro[mask])

    # Compute flux
    if mask.sum() > 0:
        flux = map.flux(xo=xo[mask], yo=yo[mask], zo=1.0, ro=ro, theta=theta[mask])
    else:
        flux = 0.0

    if np.any(np.all([flux > 0, flux < 0.8], axis=0)):

        # Save lightcurve, map coefficients, amplitudes and the design matrix
        coeff_list.append(map.y)
        amps_list.append(map.amp)
        A_list.append(
            map.design_matrix(
                xo=xo[mask], yo=yo[mask], zo=1.0, ro=ro, theta=theta[mask]
            )
        )
        lcs_true.append(flux)

        # Observation times
        times_list.append(eph_io.time.mjd)

        # Save spot data
        amp_spot1.append(a1)
        amp_spot2.append(a2)
        t_spot.append(t)

        # Save some images for visualization
        map_ = map.render(theta=theta[mask][::50])
        map_imgs.append(map_)

In [ ]:
# Plot flux
fig, ax = plt.subplots(figsize=(15, 4))
ax.plot(np.concatenate(lcs_obs), "C0.")
# ax.set_xlim(0, 10000)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(t_spot, amp_spot1, "C0o", label="spot1")
ax.plot(t_spot, amp_spot2, "C1o", label="spot2")
ax.grid()
ax.legend()

np.save("t_spot.npy", np.array(t_spot))
np.save("spot1_amp.npy", amp_spot1)
np.save("spot2_amp.npy", amp_spot2)

In [ ]:
# Save light curves
with open("lcs_true.pkl", "wb") as file:
    pkl.dump(lcs_true, file)
with open("times.pkl", "wb") as file:
    pkl.dump([times_list - times_list[0][0]], file)

# Save the design matrices
with open("design_matrices.pkl", "wb") as file:
    pkl.dump([m[:, :81] for m in A_list], file)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
map.obl = 0.0
map.show(ax=ax, image=np.concatenate(map_imgs), projection="ortho")

In [ ]:
# Construct an (N, L) matrix of coeff_list, one for each lightcurve
Y = np.vstack(coeff_list).T

# set Y_00 for each lightcurve
Y[0, :] = np.array(amps_list)

# Run PCA on the matrix
from sklearn.decomposition import PCA

n_components = 4
pca = PCA(n_components)
res = pca.fit(Y.T)
res.explained_variance_

In [ ]:
from scipy.linalg import block_diag

P = res.components_.T
print(np.shape(P))

# Save matrices to file
np.save("true_Y.npy", Y[:81, :])
np.save("true_P.npy", P[:81, :])

In [ ]:
P[0, :]

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(Y[:81, :])
ax.set_title("Matrix of SH coeff_list for each lightcurve (up to order 8)")

In [ ]:
map = starry.Map(ydeg=ydeg)

fig, ax = plt.subplots(n_components, 1, figsize=(12, 18), sharex=True)

for i in range(n_components):

    map.y[1:] = P[1:, i]
    map.amp = P[0, i]

    img = map.render(projection="rect")
    extent = (-180, 180, -90, 90)
    im = ax[i].imshow(img, origin="lower", extent=extent, cmap="plasma")
    ax[i].set_yticks([-90, -60, -30, 0, 30, 60, 90])
    ax[i].set_xticks([-180, -90, 0, 90, 180])
    fig.colorbar(im, ax=ax[i])

ax[0].set_title("PCA components for the stacked maps (ydeg=30)")

In [ ]:
# Plot the power spectrum
np.shape(Y)

In [ ]:
ps = []

idx = 1
for l in range(30):
    start = int((l + 1) ** 2)
    end = start + (2 * l + 1)
    ps.append(np.sum(P[start:end, idx] ** 2))

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(ps, "ko-")
ax.set_xlabel("l")
ax.grid()
ax.set_title(f"PCA component {idx}")